# E.4 Calculations

In [25]:
import sympy
from sympy import *
from sympy.physics.vector.printing import vlatex
from IPython.display import Math, display

init_printing()

def dotprint(expr):
    display(Math(vlatex(expr)))

### Define Symbols and Configuration Variables

In [26]:
# Time symbol
t = symbols('t')

In [27]:
# Generalized coordinates

# TODO: Change this!
z = symbols('z', cls=Function)
z = z(t)
theta = symbols('\\theta', cls=Function)
theta = theta(t)

In [28]:
# Derivatives of Generalized Coordinates

# TODO: Change this!
z_dot = z.diff(t)
z_ddot = z_dot.diff(t)
theta_dot = theta.diff(t)
theta_ddot = theta_dot.diff(t)

In [29]:
# Other symbols (mass, forces, damping coefficients, torques...)

# TODO: Change this!
m1, m2, F, el, g, ze, j = symbols('m_1, m_2, F, \\ell, g, z_e, j')

### Kinetic Energy

#### Translational Kinetic Energy

In [30]:
# TODO: Change this!
K_T = 1/6*el**2*m2*theta_dot**2 + 1/2*m1*z**2*theta_dot**2 + 1/2*m1*z_dot**2

dotprint(K_T)

<IPython.core.display.Math object>

#### Rotational Kinetic Energy (ignore)

In [31]:
# TODO: Change this!
K_R = 0

dotprint(K_R)

<IPython.core.display.Math object>

#### Total Kinetic Energy

In [32]:
K = K_T #+ K_R

dotprint(K)

<IPython.core.display.Math object>

### Potential Energy

In [33]:
# TODO: Change this!
P = m1*g*z*sin(theta) + m2*g*el*sin(theta)

dotprint(P)

<IPython.core.display.Math object>

### Define the Lagrangian

For this class, the Lagrangian will in general be a scalar value equal to the difference between kinetic and potential energy.

In [34]:
L = K - P

dotprint(L)

<IPython.core.display.Math object>

### Define the Forces

#### Generalized Forces

The forces will form a vector with the same length as the number of configuration variables. 

The first entry corresponds to the first configuration variable, the second entry to the second, etc.

Each entry should contain the generalized forces acting on the corresponding configuration variable.

In [35]:
# TODO: Change this!
tau = Matrix([0, F*el*cos(theta)])

dotprint(tau)

<IPython.core.display.Math object>

#### Non-conservative Forces (e.g. Damping)

Similar to the generalized forces, this will be a vector with the same length as the number of configuration variables.

Each entry should contain the nonconservative forces acting on the corresponding configuration variable.

In [36]:
# TODO: Change this!
B = Matrix([0, 0])
q_dot = Matrix([0])

Bqdot = B @ q_dot

dotprint(Bqdot)

<IPython.core.display.Math object>

In [37]:
F_total = tau + Bqdot

dotprint(F_total)

<IPython.core.display.Math object>

### Compute the Lagrangian Derivatives / Partial Derivatives

The following function computes these two terms:

* $\frac{d}{dt} \left(\frac{\partial L(q, \dot{q})}{\partial \dot{q}} \right)$
* $\frac{\partial L(q,\dot{q})}{\partial q}$

Things to note:

* `q` and `q_dot` should be Python tuples containing your configuration variables (e.g. `(x, theta)`)
* The function returns a Python list. You probably want to wrap the output in a Sympy `Matrix()`. For example,

```python
out = derive_Lagrangian(...)
out = Matrix(out)
```

In [38]:
def derive_Lagrangian(L, z, theta, z_dot, theta_dot):
    term_1 = (sympy.tensor.derive_by_array(L, z_dot)).diff(t)
    term_2 = sympy.tensor.derive_by_array(L, z)
    term_3 = (sympy.tensor.derive_by_array(L, theta_dot)).diff(t)
    term_4 = sympy.tensor.derive_by_array(L, theta)
    ans = Matrix([term_1 - term_2, term_3 - term_4])
    return ans

### Get the final Euler Lagrange Equations of Motion

Remember that the Euler Lagrange Equations used in class are:

$$
\underbrace{\frac{d}{dt} \left(\frac{\partial L(q, \dot{q})}{\partial \dot{q}} \right) - \frac{\partial L(q,\dot{q})}{\partial q}}_\text{Left Hand Side (LHS)} = \underbrace{\tau - B \dot{q}}_\text{Right Hand Side (RHS)}.
$$

In Sympy, we use the `Eq(LHS, RHS)` class to get a symbolic equation $\text{LHS} = \text{RHS}$.

Both the left and right expressions should be Sympy vectors (i.e. Matrices with 1 column). 

The length of the left and right vectors must both be equal to the number of configuration variables.

In [39]:
# TODO: Change this!
LHS = derive_Lagrangian(L, z, theta, z_dot, theta_dot)
RHS = F_total

Euler_Lagrange = Eq(LHS, RHS)

dotprint(simplify(Euler_Lagrange))

<IPython.core.display.Math object>

# Linearizing stuff...

In [40]:
solve_dict = solve(Euler_Lagrange, (z_ddot, theta_ddot), simplify = True, dict = True)[0]
dotprint(solve_dict)

<IPython.core.display.Math object>

In [41]:
state = Matrix([z, theta, z_dot, theta_dot])

In [42]:
z_ddot_expr = solve_dict[z_ddot]
theta_ddot_expr = solve_dict[theta_ddot]
state_derive = Matrix([z_dot, theta_dot, z_ddot_expr, theta_ddot_expr])
dotprint(state_derive)

<IPython.core.display.Math object>

In [43]:
A = state_derive.jacobian(state)
dotprint(A)

<IPython.core.display.Math object>

In [44]:
inputs = Matrix([[F]])

In [45]:
B = state_derive.jacobian(inputs)
dotprint(B)

<IPython.core.display.Math object>

In [46]:
subs_dict = {
    z: state[0],
    theta: state[1],
    z_dot: state[2],
    theta_dot: state[3]
}

In [47]:
f_expr = state_derive.subs(subs_dict)
dotprint(f_expr)

<IPython.core.display.Math object>

this is for substitutions into the A matrix

In [48]:
equilib_eq = Eq(f_expr, Matrix([0,0,0,0]))

eq_solve_dict = solve(equilib_eq, (state[0], state[1], state[2], state[3], tau), simplify=True, dict=True)[0]
dotprint(eq_solve_dict)

<IPython.core.display.Math object>

In [49]:

A_lin = A.subs({z: ze, theta: 0, z_dot: 0, theta_dot: 0, F: (g*m1*ze)/el + g*m2})
A_lin

⎡                                        0                                     ↪
⎢                                                                              ↪
⎢                                        0                                     ↪
⎢                                                                              ↪
⎢                                        0                                     ↪
⎢                                                                              ↪
⎢                                    ⎛                  ⎛       g⋅m₁⋅zₑ⎞       ↪
⎢                          2.0⋅m₁⋅zₑ⋅⎜-\ell⋅g⋅m₂ + \ell⋅⎜g⋅m₂ + ───────⎟ - g⋅m ↪
⎢        3.0⋅g⋅m₁                    ⎝                  ⎝        \ell  ⎠       ↪
⎢- ───────────────────── - ─────────────────────────────────────────────────── ↪
⎢      2               2                                                 2     ↪
⎢  \ell ⋅m₂ + 3.0⋅m₁⋅zₑ             ⎛                      2           2⎞      ↪
⎣                           

In [50]:

B_lin = B.subs({z: ze, theta: 0, z_dot: 0, theta_dot: 0, F: (g*m1*ze)/el + g*m2})
B_lin

⎡          0          ⎤
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎢      3.0⋅\ell       ⎥
⎢─────────────────────⎥
⎢    2               2⎥
⎣\ell ⋅m₂ + 3.0⋅m₁⋅zₑ ⎦